In this notebook I will walk through the basics of modeling data using Fact and Dimension tables. We will create both Fact and Dimension tables and show how this is a basic element of the Star Schema.

In [4]:
import psycopg2

Create a connection to the cursor, and set autocommit=True

In [2]:
try:
    conn = psycopg2.connect("dbname=udacity")
except psycopg2.Error as e:
    print("Error: Could not connect to database")
    print(e)

In [3]:
try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print("Error: Could not get cursor to the database")
    print(e)
conn.set_session(autocommit=True)

### Let's imagine we work at an online Music Store. There will be many tables in our database but let's just focus on 4 tables around customer purchases.

Table Name (Fact Table): customer_transactions<br>
column 0: Customer Id<br>
column 1: Store Id<br>
column 2: Spent<br>
<br>
Table Name: Customer<br>
column 0: Customer Id<br>
column 1: Name<br>
column 2: Rewards<br>
<br>
Table Name: store<br>
column 0: store Id<br>
column 1: state<br>
<br>
Table Name: items_purchased<br>
column 0: customer Id<br>
column 1: item name<br>
<br>

![title](star_schema.png)

#### From this representation, we can already start to see the makings of a "STAR". We have one fact table (the center of the star) and 3 dimension tables that are coming from it. 

Let's create the Fact Table and insert the data into the table

In [5]:
try:
    cur.execute("CREATE TABLE IF NOT EXISTS customer_transactions (customer_id int, store_id int, spent numeric);")
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

In [6]:
try:
    cur.execute("INSERT INTO customer_transactions (customer_id, store_id, spent) VALUES (%s, %s, %s)", \
                (1, 1, 20.50))
except psycopg2.Error as e:
    print("Error: Inserting rows")
    print(e)
    
try:
    cur.execute("INSERT INTO customer_transactions (customer_id, store_id, spent) VALUES (%s, %s, %s)", \
                (2, 1, 35.21))
except psycopg2.Error as e:
    print("Error: Inserting rows")
    print(e)

Now let's create our dimension tables and insert the data to those tables:

In [7]:
try:
    cur.execute("CREATE TABLE IF NOT EXISTS customer (customer_id int, name varchar, rewards boolean);")
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

In [8]:
try:
    cur.execute("INSERT INTO customer (customer_id, name, rewards) VALUES (%s, %s, %s)", \
                (1, "Amanda", True))
except psycopg2.Error as e:
    print("Error: Inserting rows")
    print(e)
    
try:
    cur.execute("INSERT INTO customer (customer_id, name, rewards) VALUES (%s, %s, %s)", \
                (2, "Toby", False))
except psycopg2.Error as e:
    print("Error: Inserting rows")
    print(e)

In [9]:
try:
    cur.execute("CREATE TABLE IF NOT EXISTS store (store_id int, state varchar);")
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

In [10]:
try:
    cur.execute("INSERT INTO store (store_id, state) VALUES (%s, %s)", \
                (1, "CA"))
except psycopg2.Error as e:
    print("Error: Inserting rows")
    print(e)
    
try:
    cur.execute("INSERT INTO store (store_id, state) VALUES (%s, %s)", \
                (2, "WA"))
except psycopg2.Error as e:
    print("Error: Inserting rows")
    print(e)

In [11]:
try:
    cur.execute("CREATE TABLE IF NOT EXISTS items_purchased (customer_id int, item_number int, item_name varchar);")
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

In [13]:
try:
    cur.execute("INSERT INTO items_purchased (customer_id, item_number, item_name) VALUES (%s, %s, %s)", \
                (1, 1, "Rubber Soul"))
except psycopg2.Error as e:
    print("Error: Inserting rows")
    print(e)
    
try:
    cur.execute("INSERT INTO items_purchased (customer_id, item_number, item_name) VALUES (%s, %s, %s)", \
                (2, 3, "Let It Be"))
except psycopg2.Error as e:
    print("Error: Inserting rows")
    print(e)

We can do a variety of queries on this data easily because of utilizing the fact/dimension and Star Schema

- Query 1: Find all the customers that spent more than 30 dollars, who are they, what did they buy and if they are a rewards member
- Query 2: How much did Store 1 sell

Query 1: Find all the customers that spent more than 30 dollars, who are they, what did they buy and if they are a rewards member

![title](star_schema.png)

In [18]:
try:
    cur.execute("SELECT name, item_name, rewards FROM ((customer \
                JOIN customer_transactions ON customer.customer_id = customer_transactions.customer_id) \
                JOIN items_purchased ON \
                customer_transactions.customer_id = items_purchased.customer_id) \
                WHERE spent > 30")
    
except psycopg2.Error as e:
    print("Error: select *")
    print(e)
    
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

('Toby', 'Let It Be', False)


Query 2: How much did Store 1 sell

In [17]:
try:
    cur.execute("SELECT store_id, SUM(spent) FROM customer_transactions GROUP BY store_id")
    
except psycopg2.Error as e:
    print("Error: select *")
    print(e)
    
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

(1, Decimal('55.71'))


### Summary: What we can see here is this elegant schema that we were able to get "facts/metrics" from our fact table (how much each store sold), and also information about our customers that will allow us to do more in-depth analytics to get answers to business questions by utilizing our fact and dimension tables.

In [19]:
# Close cursor and connection:
cur.close()
conn.close()